---
## Python Packages & Directories
---

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import (division, print_function, absolute_import,
                        unicode_literals)
import  sys, os, glob, time, IPython

import astropy.constants as const
import astropy.units as u
from astropy.io import fits
from astropy.io import ascii
# from astropy.table import Table
# from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, EarthLocation

# from PyAstronomy import pyasl

import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec

import numpy as np
import scipy as sp
import pandas as pd

import seaborn as sns
sns.set_palette("colorblind")

# from smh import Session

import spag.read_data as rd
import spag.coordinates as coord
# import alexmods.read_data as rd


script_dir = "/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[:-1]) + "/"
# script_dir = os.path.dirname(os.path.realpath(__file__))+"/"
data_dir = script_dir+"../data/"


        Use @ instead. [gala.coordinates.sgr]
        Use @ instead. [gala.coordinates.orphan]
        Use @ instead. [gala.coordinates.magellanic_stream]


---
## Read-in Chiti+2018
---

In [2]:
def load_chiti2018(combine_tables=True):
    """
    Loads the data from Chiti et al. (2018) for the MagE and M2FS measurements.
    This function reads in the data from the two tables (table5/MagE and table6/M2FS) and
    returns them as pandas DataFrames. By default, the two tables are combined into a single
    DataFrame.
    """

    # Extract the limit columns
    def separate_limit_columns(df):
        """
        Separate the limit columns from the value columns in the given DataFrame.
        """
        limit_cols = [col for col in df.columns if col.startswith('l_')]
        for l_col in limit_cols:
            value_col = l_col.replace('l_', '')  # Corresponding value column

            mask_actual = (df[l_col] != '<') & (df[l_col] != '>')
            mask_lower = df[l_col] == '>'
            mask_upper = df[l_col] == '<'

            df[f"{value_col}_real"] = np.where(mask_actual, df[value_col], np.nan) # Actual values
            df[f"{value_col}_ll"] = np.where(mask_lower, df[value_col], np.nan) # Lower limit
            df[f"{value_col}_ul"] = np.where(mask_upper, df[value_col], np.nan) # Upper limit

            # Drop the original limit column & value column
            df = df.drop(columns=[value_col])
            df = df.drop(columns=[l_col])
            df = df.rename(columns={f"{value_col}_real": value_col})
        return df

    if combine_tables:

        ## Load the combined table (created by Alex Yelland)
        chiti18_df = pd.read_csv('/Users/ayelland/Research/metal-poor-stars/spag/spag/data/abundance_tables/chiti2018/chiti2018_sculptor.csv', comment='#', header=0)
        
        ## Add columns and extract the limit columns
        chiti18_df = separate_limit_columns(chiti18_df)
        df_cols_reorder = [
            'Reference','Ref','Name','Loc','Type','Sci_key','RA_hms','DEC_dms','RA_deg','DEC_deg',
            'logg','Teff','[Ba/H]','Slit',
            'A(C)','A(C)_ll','A(C)_ul','e_A(C)',
            '[Fe/H]','[Fe/H]_ll','[Fe/H]_ul','e_[Fe/H]',
            '[C/Fe]','[C/Fe]_ll','[C/Fe]_ul','e_[C/Fe]','[C/Fe]c',
            '[C/Fe]f','[C/Fe]f_ll','[C/Fe]f_ul','e_[C/Fe]f',
        ]
        chiti18_df = chiti18_df[df_cols_reorder]

        ## Fill the NaN values in the RA and DEC columns
        for idx, row in chiti18_df.iterrows():
            if pd.isna(row['RA_deg']) and pd.notna(row['RA_hms']):  # Ensure RA_hms is not NaN
                row['RA_deg'] = coord.ra_hms_to_deg(str(row['RA_hms']), precision=6)
                chiti18_df.at[idx, 'RA_deg'] = row['RA_deg']

            if pd.isna(row['DEC_deg']) and pd.notna(row['DEC_dms']):  # Ensure DEC_dms is not NaN
                row['DEC_deg'] = coord.dec_dms_to_deg(str(row['DEC_dms']), precision=6)
                chiti18_df.at[idx, 'DEC_deg'] = row['DEC_deg']

            if pd.isna(row['RA_hms']) and pd.notna(row['RA_deg']):  # Ensure RA_deg is not NaN
                row['RA_hms'] = coord.ra_deg_to_hms(row['RA_deg'], precision=2)
                chiti18_df.at[idx, 'RA_hms'] = row['RA_hms']

            if pd.isna(row['DEC_dms']) and pd.notna(row['DEC_deg']):  # Ensure DEC_deg is not NaN
                row['DEC_dms'] = coord.dec_deg_to_dms(row['DEC_deg'], precision=2)
                chiti18_df.at[idx, 'DEC_dms'] = row['DEC_dms']

        return chiti18_df

    else:
        ## Table 5: MagE Measurements
        mage_df = pd.read_csv('/Users/ayelland/Research/metal-poor-stars/spag/spag/data/abundance_tables/chiti2018/table5.csv', comment='#', header=None)
        mage_df.columns = [
            'ID', 'f5_ID', 'Slit', 'logg', 'Teff', 'l_[Fe/H]KP', '[Fe/H]KP', 'e_[Fe/H]KP', 
            'l_A(C)', 'A(C)', 'e_A(C)', 'l_[C/Fe]', '[C/Fe]', 'e_[C/Fe]', '[C/Fe]c',
            'l_[C/Fe]f','[C/Fe]f', 'e_[C/Fe]f', '[Ba/H]', 'RA_deg', 'DEC_deg'
        ]
        mage_df['Reference'] = 'Chiti+2018_MagE'
        mage_df['Ref'] = 'CHI18'
        mage_df = separate_limit_columns(mage_df)
        mage_cols_reorder = [
            'Reference','Ref','ID','f5_ID','RA_deg','DEC_deg','Slit','logg','Teff',
            '[Fe/H]KP','[Fe/H]KP_ll','[Fe/H]KP_ul','e_[Fe/H]KP',
            'A(C)','A(C)_ll','A(C)_ul','e_A(C)',
            '[C/Fe]','[C/Fe]_ll','[C/Fe]_ul','e_[C/Fe]',
            '[C/Fe]c','[C/Fe]f','[C/Fe]f_ll','[C/Fe]f_ul','e_[C/Fe]f','[Ba/H]']
        mage_df = mage_df[mage_cols_reorder]

        ## Table 6: M2FS Measurements
        m2fs_df = pd.read_csv('/Users/ayelland/Research/metal-poor-stars/spag/spag/data/abundance_tables/chiti2018/table6.csv', comment='#', header=None)
        m2fs_df.columns = [
            'Type', 'ID', 'f6_ID', 'RA_hms', 'DEC_dms', 'logg', 'Teff',
            'l_[Fe/H]', '[Fe/H]', 'e_[Fe/H]', 
            'l_[C/Fe]', '[C/Fe]', 'e_[C/Fe]', '[C/Fe]c', 'l_[C/Fe]f', '[C/Fe]f'
        ]
        m2fs_df['Reference'] = 'Chiti+2018_M2FS'
        m2fs_df['Ref'] = 'CHI18'
        m2fs_df = separate_limit_columns(m2fs_df)
        m2fs_cols_reorder = [
            'Reference','Ref','ID','f6_ID','RA_hms','DEC_dms','Type','logg','Teff',
            '[Fe/H]','[Fe/H]_ll','[Fe/H]_ul','e_[Fe/H]',
            '[C/Fe]','[C/Fe]_ll','[C/Fe]_ul','e_[C/Fe]',
            '[C/Fe]c','[C/Fe]f','[C/Fe]f_ll','[C/Fe]f_ul']
        m2fs_df = m2fs_df[m2fs_cols_reorder]

        return mage_df, m2fs_df


---
## Create `load_sculptor()` function
---

In [30]:
def load_chiti2018_extract_carbon_data_original(chiti2018_df):
    """
    Extracts the [Fe/H], [C/Fe], [C/Fe]_ul, and [C/Fe]_ll values from the
    Chiti et al. (2018) DataFrame. The function accounts for the different
    column names used in the MagE and M2FS tables.

    chi18_df: pandas DataFrame
        MagE or M2FS DataFrame containing the data from Chiti et al. (2018).

    Returns:
        feh: list
            Extracted [Fe/H] values.
        feh_ul: list
            Extracted [Fe/H] upper limit values.
        feh_ll: list
            Extracted [Fe/H] lower limit values.
        cfe: list
            Extracted [C/Fe] values.
        cfe_ul: list
            Extracted [C/Fe] upper limit values.
        cfe_ll: list
            Extracted [C/Fe] lower limit values.
    """
    feh = []
    feh_ul = []
    feh_ll = []
    cfe = []
    cfe_ul = []
    cfe_ll = []
    
    feh_str = '[Fe/H]KP' if '[Fe/H]KP' in chiti2018_df.columns else '[Fe/H]'

    for idx, row in chiti2018_df.iterrows():
        if pd.isna(row[feh_str]):
            if pd.isna(row[feh_str+'_ul']):
                if pd.isna(row[feh_str+'_ll']):
                    feh.append(np.nan)
                    feh_ul.append(np.nan)
                    feh_ll.append(np.nan)
                    cfe.append(np.nan)
                    cfe_ul.append(np.nan)
                    cfe_ll.append(np.nan)
                else:
                    feh.append(np.nan)
                    feh_ul.append(np.nan)
                    feh_ll.append(row[feh_str+'_ll'])

                    if pd.isna(row['[C/Fe]']):
                        if pd.isna(row['[C/Fe]_ul']):
                            if pd.isna(row['[C/Fe]_ll']):
                                feh[-1] = np.nan
                                feh_ul[-1] = np.nan
                                feh_ll[-1] = np.nan
                                cfe.append(np.nan)
                                cfe_ul.append(np.nan)
                                cfe_ll.append(np.nan)
                            else:
                                cfe.append(np.nan)
                                cfe_ul.append(np.nan)
                                cfe_ll.append(row['[C/Fe]_ll'])
                        else:   
                            cfe.append(np.nan)
                            cfe_ul.append(np.nan)
                            cfe_ll.append(row['[C/Fe]_ul'])
                    else:
                        cfe.append(np.nan)
                        cfe_ul.append(np.nan)
                        cfe_ll.append(row['[C/Fe]'])
            else:
                feh.append(np.nan)
                feh_ul.append(row[feh_str+'_ul'])
                feh_ll.append(np.nan)
                if pd.isna(row['[C/Fe]']):
                    if pd.isna(row['[C/Fe]_ul']):
                        if pd.isna(row['[C/Fe]_ll']):
                            feh[-1] = np.nan
                            feh_ul[-1] = np.nan
                            feh_ll[-1] = np.nan
                            cfe.append(np.nan)
                            cfe_ul.append(np.nan)
                            cfe_ll.append(np.nan)
                        else:
                            cfe.append(np.nan)
                            cfe_ul.append(row['[C/Fe]_ll'])
                            cfe_ll.append(np.nan)
                    else:   
                        cfe.append(np.nan)
                        cfe_ul.append(row['[C/Fe]_ul'])
                        cfe_ll.append(np.nan)
                else:
                    cfe.append(np.nan)
                    cfe_ul.append(row['[C/Fe]'])
                    cfe_ll.append(np.nan)
        else:
            feh.append(row[feh_str])
            feh_ul.append(np.nan)
            feh_ll.append(np.nan)
            if pd.isna(row['[C/Fe]']):
                if pd.isna(row['[C/Fe]_ul']):
                    if pd.isna(row['[C/Fe]_ll']):
                        feh[-1] = np.nan
                        feh_ul[-1] = np.nan
                        feh_ll[-1] = np.nan
                        cfe.append(np.nan)
                        cfe_ul.append(np.nan)
                        cfe_ll.append(np.nan)
                    else:
                        cfe.append(row['[C/Fe]_ll'])
                        cfe_ul.append(np.nan)
                        cfe_ll.append(np.nan)
                else:   
                    cfe.append(row['[C/Fe]_ul'])
                    cfe_ul.append(np.nan)
                    cfe_ll.append(np.nan)
            else:
                cfe.append(row['[C/Fe]'])
                cfe_ul.append(np.nan)
                cfe_ll.append(np.nan)

    print("len:  ", len(feh), len(feh_ul), len(feh_ll), len(cfe), len(cfe_ul), len(cfe_ll))
    print("real: ", np.sum(~np.isnan(feh)), np.sum(~np.isnan(cfe)))
    print("ulim: ", np.sum(~np.isnan(feh_ul)), np.sum(~np.isnan(cfe_ul)))
    print("llim: ", np.sum(~np.isnan(feh_ll)), np.sum(~np.isnan(cfe_ll)))

    return feh, feh_ul, feh_ll, cfe, cfe_ul, cfe_ll

chiti18_df = load_chiti2018(combine_tables=True)
feh, feh_ul, feh_ll, cfe, cfe_ul, cfe_ll = load_chiti2018_extract_carbon_data_original(chiti18_df)

chiti18_df_updated = chiti18_df.copy()
chiti18_df_updated['FeH'] = feh
chiti18_df_updated['FeH_ll'] = feh_ll
chiti18_df_updated['FeH_ul'] = feh_ul
chiti18_df_updated['C/Fe'] = cfe
chiti18_df_updated['C/Fe_ll'] = cfe_ll
chiti18_df_updated['C/Fe_ul'] = cfe_ul

chiti18_df_updated.to_csv('/Users/ayelland/Research/metal-poor-stars/project/carbon-project/updated_df.csv', index=False)



len:   127 127 127 127 127 127
real:  108 108
ulim:  0 0
llim:  0 0


In [40]:
def load_sculptor():

    sculptor_refs = [
        'CHI18','GEI05','HIL19','JAB15','KIR12',#'REI20',
        'SIM15','SKU15','SKU17','SKU19','SKU24','SHE03',
        'TAF10'
        # 'REI20' could be included, but it lacks carbon abundances and does not have DW identifiers in jinabase
    ]

    ## JINAbase
    # -------------------------------------------------- #
    jinabase = rd.load_jinabase(priority=1)

    ## Filter JINAbase for Sculptor (by Star Name)
    jinabase_nan = jinabase[jinabase['Name'].isna()]  # Rows where 'Name' is NaN
    jinabase_non_nan = jinabase[jinabase['Name'].notna()]  # Rows where 'Name' is not NaN
    jinabase_sculptor1 = jinabase_non_nan[jinabase_non_nan['Name'].str.lower().str.contains('scl')]

    ## Filter JINAbase for Sculptor (by Reference)
    sculptor_refs = [ref for ref in sculptor_refs if ref not in jinabase_sculptor1['Ref'].unique()]
    jinabase_sculptor2 = jinabase[jinabase['Ref'].isin(sculptor_refs)]

    ## References Remaining
    sculptor_refs = [ref for ref in sculptor_refs if ref not in jinabase_sculptor2['Ref'].unique()]

    ## Concatenate the DataFrames
    jinabase_sculptor = pd.concat([jinabase_sculptor1, jinabase_sculptor2], ignore_index=True)


    ## Manually add specific references
    # -------------------------------------------------- #

    ## Chiti+2018
    chiti18_df = load_chiti2018(combine_tables=True)

    chiti18_df['epsc'] = chiti18_df['A(C)']
    chiti18_df['ulc'] = chiti18_df['A(C)_ul']
    chiti18_df['llc'] = chiti18_df['A(C)_ll']
    chiti18_df['e_epsc'] = chiti18_df['A(C)_ul']

    chiti18_df.drop(columns=[
        'logg',
        '[Ba/H]',
        'Slit',
        'A(C)',
        'A(C)_ll',
        'A(C)_ul',
        'e_A(C)',
        # '[Fe/H]_ll',
        # '[Fe/H]_ul',
        # 'e_[Fe/H]',
        # '[C/Fe]_ll',
        # '[C/Fe]_ul',
        # 'e_[C/Fe]',
        # '[C/Fe]c',
        # '[C/Fe]f',
        # '[C/Fe]f_ll',
        # '[C/Fe]f_ul',
        # 'e_[C/Fe]f'
    ], inplace=True)

    # chiti18_df.to_csv('/Users/ayelland/Research/metal-poor-stars/project/carbon-project/chiti2018_sculptor.csv', index=False)

    ## Skuladottir+2017
    ## Skuladottir+2024
    ## Frebel+2010
    ## Jacobson

    ## Combine the DataFrames
    # -------------------------------------------------- #
    sculptor_df = pd.concat([jinabase_sculptor, chiti18_df], ignore_index=True, sort=False)
    sculptor_df.to_csv('/Users/ayelland/Research/metal-poor-stars/project/carbon-project/sculptor_df.csv', index=False)

    return sculptor_df



sculptor_df = load_sculptor()
display(sculptor_df.shape[0])


sculptor_df_carbon = sculptor_df[sculptor_df['[C/Fe]'].notna() | sculptor_df['ulc'].notna()]

# sculptor_df_carbon = sculptor_df[sculptor_df['[C/Fe]'] | sculptor_df['ulc']]
display(sculptor_df_carbon.shape[0])

for ref in list(sculptor_df['Reference'].unique()):
    print(f"{ref}: {sculptor_df[sculptor_df['Reference'] == ref].shape[0]}")
# display(sculptor_df_carbon)



238

109

Geisler+2005: 4
Hill+2019: 87
JAB15: 5
KIR12: 1
SHE03: 5
SIM15: 5
Skuladottir+2015: 1
Skuladottir+2019: 1
TAF10: 2
Chiti+2018_M2FS: 100
Chiti+2018_MagE: 27
